# Covid19 World Data, Interactive Dashboard

COVID-19 Data repository by John Hopkins CSSE: https://github.com/CSSEGISandData/COVID-19
(Data is updated every 24 hours)

In [139]:
from __future__ import print_function

import pandas as pd
import numpy as np


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import Image


import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import plotly.graph_objects as go
plt.style.use('ggplot')

In [140]:
# loading data right from the source:

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')


In [141]:
# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [142]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

### Spread on the world map (zoom-in/zoom-out, click on a country to view total number of confirmed cases)

In [143]:
# creating world map using Map class
world_map = folium.Map(location=[11,0], tiles='Stamen Terrain', zoom_start=2, max_zoom = 6, min_zoom = 2)
# iterate over all the rows of confirmed_df to get the lat/long
for i in range(0,len(confirmed_df)):
    number_of_cases=int(confirmed_df.iloc[i,-1])
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='#ff0080',
        fill_color='blue',
        popup=str(number_of_cases)
    ).add_to(world_map)
    
world_map

In [144]:
print('Total confirmed cases: ',confirmed_total)
print('\n')
print('Total recoveries: ',recovered_total)
print('\n')
print('Total deaths: ',deaths_total)
print('\n')
print('Total active cases: ',active_total)

Total confirmed cases:  5215253


Total recoveries:  2059228


Total deaths:  338265


Total active cases:  2857452


## *....Apologies for the delayed display, please wait while the server loads data*

In [145]:
#function to highlight required columns
def highlight_col(x):
    r = 'background-color: grey'
    p = 'background-color: #a448ff'
    g = 'background-color: #bbec8e'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = p
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    return df1

### Top confirmed cases by country (input number of countries you want displayed)

In [146]:
#function to display number of worst hit countries you want to see
def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed',ascending= False).reset_index(drop=True).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

<function __main__.show_latest_cases(n)>

### Worst-hit countries in terms of cases (Use slider to change the number of countries you want displayed)

In [147]:
sorted_country_df=country_df.sort_values('confirmed',ascending=False)

In [148]:
def bubble_chart(n):
    if n > 0:
        fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
                   hover_name="country", size_max=60)
        fig.update_layout(
        title=str(n) +" Worst hit countries (Cases)",
        xaxis_title="Countries",
        yaxis_title="Confirmed Cases",
        width = 900
        )
        fig.show()

        

interact(bubble_chart, n=10)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart(n)>

### Top 10 countries: number of deaths

In [149]:
sorted_country_deaths_df=sorted_country_df.sort_values('deaths',ascending=False)
sorted_country_deaths_df=sorted_country_deaths_df.reset_index(drop=True)

In [150]:
px.bar(
    sorted_country_deaths_df.head(10),
    x = "country",
    y = "deaths",
    title= "Countries with highest number of deaths",
    color_discrete_sequence=['#e30072'], 
    height=500,
    width=1000
)

### Top 10 countries: death rates

In [151]:
sorted_country_df['death_rates']=sorted_country_df['deaths']/sorted_country_df['confirmed']
sorted_country_deathrates_df=sorted_country_df.sort_values('death_rates',ascending=False)
sorted_country_deathrates_df=sorted_country_deathrates_df.reset_index(drop=True)

In [152]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "death_rates",
    title= "Countries with the highest death rates (**Ratio of Total Deaths : Total Confirmed Cases)",
    color_discrete_sequence=["#b93dc2"], 
    height=500,
    width=800
)

### Confirmed cases, deaths and recoveries by day for a select country (Input name of country you want to view)

In [170]:
def plot_cases_of_a_country(country):
    labels = ['recovered','confirmed', 'deaths']
    colors = ['green','blue', 'red']
    mode_size = [8,6, 8]
    line_size = [5, 4, 5]
    
    df_list = [recovered_df,confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 23:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 23:].columns))
            y_data = np.sum(np.asarray(df[df['country'].str.lower() == country.lower()].iloc[:,4:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="Covid 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [171]:
interactive(plot_cases_of_a_country,country='world')

interactive(children=(Text(value='world', description='country'), Output()), _dom_classes=('widget-interact',)…

In [155]:
# widgets.ColorPicker(
#     concise=False,
#     description='Pick a color',
#     value='blue'
# )

Developed by <a href="https://www.jacsons.co.ke/">jacsons.co.ke</a>